In [7]:
# from __future__ import annotations
# import numpy as np
# import pandas as pd
# from typing import Optional, Tuple, Union, Sequence, List
# from abc import ABC, abstractmethod
# import matplotlib.pyplot as plt
# import inspect
# from survivors.external import BaseSAAdapter

# class ClassifWrapSA(BaseSAAdapter):
#     def __init__(self, model, times: int = 128, start_at_zero: bool = True):
#         self.model = model
#         self.__name__ = f"ClassifWrapSA({model.__class__.__name__})"
#         self.times = times
#         self.start_at_zero = start_at_zero
#         self._bounds: Optional[Tuple[float, float]] = None
    
#     def __call__(self, **kwargs):
#         self.model = self.model.__class__(**kwargs)
#         return self
        
#     def fit(self, X, y, time_col: str = "time", event_col: str = "event"):
#         t, e = self.timeWrap(y, time_col, event_col)
#         left = float(np.min(t))
#         right = float(np.max(t))
#         if not np.isfinite(left) or not np.isfinite(right) or left >= right:
#             left = 0.0
#             right = 1.0
#         self._bounds = (left, right)
#         self.model.fit(X, e.astype(int))
#         return self
#     def predict_proba(self, X):
#         proba = self._get_proba(self.model, X)
#         if proba.ndim == 1:
#             proba = np.column_stack([1 - proba, proba])
#         return proba
#     def predict_survival_function(self, X, times = None):
#         assert self._bounds is not None, "Call fit() first"
#         if times is not None:
#             t = np.asarray(times, float)
#         else:
#             left, right = self._bounds
#             start = 0.0 if self.start_at_zero else left
#             t = np.linspace(start, right, self.times) #разбивает на равномерную сетку
#         p = self._get_proba(self.model, X)
#         S = np.tile((1.0-p)[:, None], (1,t.size)) #матрица выживаемости dim(p)*t.size
#         return S, t
#     def predict_hazard_function(self, X, times=None):
#         S, t = self.predict_survival_function(X, times)
#         eps = 1e-12
#         H = -np.log(np.clip(S, eps, 1.0))   # кумулятивная hazard, константна по t
#         return H, t
#     def predict_expected_time(self, X, times=None):
#         S, t = self.predict_survival_function(X, times)
#         return np.trapz(S, t, axis=1)
#     def predict_time(self, X):
#         n = len(X) if hasattr(X, "__len__") else self.predict_proba(X).shape[0]
#         return np.full(n, np.nan, float)
    
# class RegrWrapSA(BaseSAAdapter):
#     def __init__(self, model, times: int = 128, start_at_zero: bool = True):
#         self.model = model
#         self.__name__ = f"RegrWrapSA({model.__class__.__name__})"
#         self.times = times
#         self.start_at_zero = start_at_zero
#         self._bounds: Optional[Tuple[float, float]] = None
    
#     def __call__(self, **kwargs):
#         self.model = self.model.__class__(**kwargs)
#         return self
        
#     def fit(self, X, y, time_col: str = "time", event_col: str = "event"):
#         t, _ = self.timeWrap(y, time_col, event_col)
#         left = float(np.min(t))
#         right = float(np.max(t))
#         if not np.isfinite(left) or not np.isfinite(right) or left >= right:
#             left = 0.0
#             right = 1.0
#         self._bounds = (left, right)
#         self.model.fit(X, t.astype(float))
#         return self
#     def predict_time(self, X):
#         return np.asarray(self.model.predict(X), float)
#     def predict_survival_function(self, X, times = None):
#         assert self._bounds is not None, "Call fit() first"
#         if times is not None:
#             t = np.asarray(times, float)
#         else:
#             left, right = self._bounds
#             start = 0.0 if self.start_at_zero else left
#             t = np.linspace(start, right, self.times) #разбивает на равномерную сетку
#         T = self.predict_time(X)[:, None]
#         S = (t[None, :] < T).astype(float)
#         return S, t
#     def predict_hazard_function(self, X, times = None):
#         S, t = self.predict_survival_function(X, times)
#         eps = 1e-12
#         H = -np.log(np.clip(S, eps, 1.0))  # кумулятивная hazard
#         return H, t
#     def predict_expected_time(self, X, times=None):
#         S, t = self.predict_survival_function(X, times)
#         return np.trapz(S, t, axis=1)
#     def predict_proba(self, X):
#         S, _ = self.predict_survival_function(X)
#         p = 1.0 - S[:, -1]
#         return np.column_stack([1.0 - p, p])
    
# class SAWrapSA(BaseSAAdapter):
#     def __init__(self, model, times: int = 128, start_at_zero: bool = True):
#         self.model = model
#         self.__name__ = f"SAWrapSA({model.__class__.__name__})"
#         self.times = times
#         self.start_at_zero = start_at_zero
#         self._bounds: Optional[Tuple[float, float]] = None
#         self._kind = model.__class__.__module__
    
#     def __call__(self, **kwargs):
#         self.model = self.model.__class__(**kwargs)
#         return self
        
#     @staticmethod
#     def _make_structured_y(t, e):
#         return np.array(list(zip(e.astype(bool), t.astype(float))), dtype=[('event','?'),('time','<f8')])
    
#     def fit(self, X, y, time_col: str = "time", event_col: str = "event"):
#         t, e = self.timeWrap(y, time_col, event_col)
#         left = float(np.min(t))
#         right = float(np.max(t))
#         if not np.isfinite(left) or not np.isfinite(right) or left >= right:
#             left = 0.0
#             right = 1.0
#         self._bounds = (left, right)
#         if "lifelines" in self._kind:
#             fit_sig = inspect.signature(self.model.fit)
#             params = fit_sig.parameters
#             if "duration_col" in params: #двумерные, например CoxPHFitter
#                 df = pd.DataFrame(X if isinstance(X, pd.DataFrame) else np.asarray(X)).copy()
#                 df[time_col] = t
#                 df[event_col] = e.astype(int)
#                 self.model.fit(df, duration_col=time_col, event_col=event_col)
#             else: #одномерные, например NelsonAalenFitter
#                 if "durations" in params:
#                     self.model.fit(durations=t, event_observed=e.astype(int))
#                 else:
#                     self.model.fit(t, e.astype(int))
#         elif self._kind.startswith("sksurv.") or "sksurv" in self._kind:
#             self.model.fit(X, self._make_structured_y(t, e))
#         elif "survivors" in self._kind:
#             self.model.fit(X, y)
#         else:
#             self.model.fit(X, y)
#         return self
#     def predict_survival_function(self, X, times = None):
#         assert self._bounds is not None, "Call fit() first"
#         if times is not None:
#             t = np.asarray(times, float)
#         else:
#             left, right = self._bounds
#             start = 0.0 if self.start_at_zero else left
#             t = np.linspace(start, right, self.times) #разбивает на равномерную сетку
#         if "lifelines" in self._kind:
#             n = len(X) if hasattr(X, "__len__") else 1
#             if hasattr(self.model, "predict_survival_function"): #двумерны
#                 df = X if isinstance(X, pd.DataFrame) else pd.DataFrame(X)
#                 sf = self.model.predict_survival_function(df)
#                 if isinstance(sf, list):
#                     S = np.vstack([np.interp(t, s.index.values.astype(float), np.asarray(s).astype(float)) for s in sf])
#                 else:   #одномерные
#                     idx = sf.index.values.astype(float)
#                     S = np.vstack([np.interp(t, idx, sf.iloc[:, i].values.astype(float)) for i in range(sf.shape[1])])
#                 return S, t
#             #одномерные 
#             if hasattr(self.model, "survival_function_at_times"):
#                 s = self.model.survival_function_at_times(t)
#                 s_vals = np.asarray(s).astype(float).ravel()
#             elif hasattr(self.model, "cumulative_hazard_at_times"):
#                 H = self.model.cumulative_hazard_at_times(t)
#                 H_vals = np.asarray(H).astype(float).ravel()
#                 s_vals = np.exp(-H_vals)
#             else:
#                 raise RuntimeError("lifelines univariate model without at_times methods")
#             S = np.tile(s_vals[None, :], (n, 1))
#             return S, t
#         elif self._kind.startswith("sksurv.") or "sksurv" in self._kind:
#             s = self.model.predict_survival_function(X)
#             S = np.vstack([sf(t) for sf in s])
#             return S, t
#         elif "survivors" in self._kind:
#             if times is not None:
#                 t = np.asarray(times, float)
#             else:
#                 left, right = self._bounds
#                 start = 0.0 if self.start_at_zero else left
#                 t = np.linspace(start, right, self.times)
#             S = np.asarray(self.model.predict_at_times(X, t, mode="surv"), float)
#             if S.ndim == 1:
#                 S = S[None, :]
#             elif S.shape[1] != t.shape[0] and S.shape[0] == t.shape[0]:
#                 S = S.T
#             return S, t
#         raise RuntimeError("Model does not provide survival predictions")
        
#     def predict_hazard_function(self, X, times = None):
#         assert self._bounds is not None, "Call fit() first"
#         if "lifelines" in self._kind:
#             if times is not None:
#                 t = np.asarray(times, float)
#             else:
#                 left, right = self._bounds
#                 start = 0.0 if self.start_at_zero else left
#                 t = np.linspace(start, right, self.times) #разбивает на равномерную сетку
#             n = len(X) if hasattr(X, "__len__") else 1
#             if hasattr(self.model, "predict_cumulative_hazard"): #двумерные
#                 df = X if isinstance(X, pd.DataFrame) else pd.DataFrame(X)
#                 ch = self.model.predict_cumulative_hazard(df)
#                 if isinstance(ch, list):
#                     H = np.vstack([np.interp(t, h.index.values.astype(float), np.asarray(h).astype(float)) for h in ch])
#                 else:
#                     idx = ch.index.values.astype(float)
#                     H = np.vstack([np.interp(t, idx, ch.iloc[:, i].values.astype(float)) for i in range(ch.shape[1])])
#                 return H, t
#             if hasattr(self.model, "cumulative_hazard_at_times"): #одномерные
#                 try:
#                     H_single = self.model.cumulative_hazard_at_times(t)
#                     H_vals = np.asarray(H_single).astype(float).ravel()
#                     H = np.tile(H_vals[None, :], (n, 1))
#                     return H, t
#                 except NotImplementedError:
#                     pass
#             if hasattr(self.model, "survival_function_at_times"):
#                 try:
#                     s = self.model.survival_function_at_times(t)
#                     s_vals = np.asarray(s).astype(float).ravel()
#                     eps = 1e-12
#                     H_vals = -np.log(np.clip(s_vals, eps, 1.0))
#                     H = np.tile(H_vals[None, :], (n, 1))
#                     return H, t
#                 except NotImplementedError:
#                     pass
#             S, t = self.predict_survival_function(X, t)
#             eps = 1e-12
#             H = -np.log(np.clip(S, eps, 1.0))
#             return H, t
#         if "survivors" in self._kind:
#             if times is not None:
#                 t = np.asarray(times, float)
#             else:
#                 left, right = self._bounds
#                 start = 0.0 if self.start_at_zero else left
#                 t = np.linspace(start, right, self.times)
#             H = np.asarray(self.model.predict_at_times(X, t, mode="hazard"), float)
#             if H.ndim == 1:
#                 H = H[None, :]
#             elif H.shape[1] != t.shape[0] and H.shape[0] == t.shape[0]:
#                 H = H.T
#             return H, t

#         S, t = self.predict_survival_function(X, times) #для всех других моделей
#         eps = 1e-12
#         H = -np.log(np.clip(S, eps, 1.0))
#         return H, t
#     def predict_expected_time(self, X, times=None):
#         S, t = self.predict_survival_function(X, times)
#         return np.trapz(S, t, axis=1)
#     def predict_time(self, X):
#         S, t = self.predict_survival_function(X)
#         med = np.full(S.shape[0], np.nan, float)
#         for i, row in enumerate(S):
#             j = np.argmax(row <= 0.5)
#             if row[j] <= 0.5:
#                 med[i] = t[j]
#         return med
#     def predict_proba(self, X):
#         S, _ = self.predict_survival_function(X)
#         p = 1.0 - S[:, -1]
#         return np.column_stack([1.0 - p, p])

# def plot_many(models, X, sample: int = 0, times: Optional[Sequence[float]] = None, title="Survival curves comparison", mode: str = "survival"):
#     if times is not None:
#         t_common = np.asarray(times, float)
#     else:
#         t_max = 1.0
#         for m in models:
#             if m._bounds is not None:
#                 t_max = max(t_max, m._bounds[1])
#         t_common = np.linspace(0.0, t_max, 128)
#     X_i = X.iloc[[sample]] if isinstance(X, pd.DataFrame) else np.asarray(X)[[sample], :]
#     plt.figure(figsize=(7, 5))
#     for m in models:
#         name = type(m.model).__name__
#         if mode.lower().startswith("h"):
#             S, _ = m.predict_hazard_function(X_i, t_common)
#             ylabel = "Hazard H(t)"
#         else:
#             S, _ = m.predict_survival_function(X_i, t_common)
#             ylabel = "Survival S(t)"
#         plt.plot(t_common, S[0], label=name)
#     plt.xlabel("Time")
#     plt.ylabel(ylabel)
#     plt.title(title)
#     plt.grid(True)
#     plt.legend()
#     plt.show()


In [1]:
from survivors.external import ClassifWrapSA, RegrWrapSA, SAWrapSA

In [2]:
from survivors.experiments import grid as exp
import survivors.datasets as ds

from sklearn.metrics import mean_squared_error

mse_exp_time = lambda y_tr, y_tst, pred_time, pred_sf, pred_hf, bins: mean_squared_error(y_tst["time"], pred_time)

l_metrics = ["CI", "IBS", "AUPRC", "MSE"]
X, y, features, categ, _ = ds.load_pbc_dataset()
experim = exp.Experiments(folds=5, mode="CV+SAMPLE")
experim.add_new_metric("MSE", mse_exp_time)
experim.set_metrics(l_metrics)

In [5]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from lifelines import KaplanMeierFitter
from sksurv.linear_model import CoxPHSurvivalAnalysis
from survivors.tree import CRAID

experim.add_method(ClassifWrapSA(KNeighborsClassifier()), {"n_neighbors": [5]})
experim.add_method(ClassifWrapSA(RandomForestClassifier()), {"n_estimators": [10]})
experim.add_method(RegrWrapSA(KNeighborsRegressor()), {"n_neighbors": [5]})
experim.add_method(SAWrapSA(KaplanMeierFitter()), dict())
experim.add_method(SAWrapSA(CoxPHSurvivalAnalysis()), {"alpha": [100]})
experim.add_method(SAWrapSA(CRAID()), {"depth": [10]})
experim.add_method(CRAID, {"depth": [10]})
experim.run_effective(X, y, verbose=0, stratify_best=[])
df_results = experim.get_result()

<survivors.external.mlwrap.ClassifWrapSA object at 0x0000027ACF05A890> {'n_neighbors': [5]}
<survivors.external.mlwrap.ClassifWrapSA object at 0x0000027ACF05AF50> {'n_estimators': [10]}
<survivors.external.mlwrap.RegrWrapSA object at 0x0000027AD141A980> {'n_neighbors': [5]}
<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD5663130> {}
<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD5661A20> {'alpha': [100]}


C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\s

<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD5662AA0> {'depth': [10]}
<class 'survivors.tree.decision_tree.CRAID'> {'depth': [10]}
<survivors.external.mlwrap.ClassifWrapSA object at 0x0000027AD26B26E0> {'n_neighbors': [5]}
<survivors.external.mlwrap.ClassifWrapSA object at 0x0000027AD26B3CA0> {'n_estimators': [10]}
<survivors.external.mlwrap.RegrWrapSA object at 0x0000027AD26B3C40> {'n_neighbors': [5]}
<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD26B3400> {}
<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD26B2530> {'alpha': [100]}


C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\s

<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD26B32E0> {'depth': [10]}
<class 'survivors.tree.decision_tree.CRAID'> {'depth': [10]}
<survivors.external.mlwrap.ClassifWrapSA object at 0x0000027AD26B26E0> {'n_neighbors': [5]}
Iteration: 1/1
EXECUTION TIME OF ClassifWrapSA(KNeighborsClassifier): [0.293 0.266 0.265 0.267 0.302 0.275 0.267 0.286 0.253 0.269 0.253 0.272
 0.263 0.273 0.284 0.252 0.29  0.258 0.269 0.278], MEM [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] {'CI': [0.555, 0.513], 'IBS': [0.246, 0.241], 'AUPRC': [0.425, 0.455], 'MSE': [2425616.138, 3009287.097]}
<survivors.external.mlwrap.ClassifWrapSA object at 0x0000027AD26B3CA0> {'n_estimators': [10]}
Iteration: 1/1
EXECUTION TIME OF ClassifWrapSA(RandomForestClassifier): [0.265 0.343 0.345 0.313 0.299 0.32  0.309 0.26  0.345 0.317 0.345 0.342
 0.303 0.341 0.308 0.274 0.334 0.342 0.303 0.268], MEM [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] {'CI': [0.617, 0.614], 'IBS': [0.196, 0.186

C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\s

C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\s

C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\s

Iteration: 1/1
EXECUTION TIME OF SAWrapSA(CoxPHSurvivalAnalysis): [0.355 0.418 0.414 0.335 0.343 0.38  0.425 0.394 0.422 0.38  0.365 0.383
 0.38  0.377 0.349 0.413 0.382 0.366 0.382 0.335], MEM [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] {'CI': [0.666, 0.667], 'IBS': [0.131, 0.134], 'AUPRC': [0.691, 0.691], 'MSE': [1659305.905, 1766814.024]}
<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD26B32E0> {'depth': [10]}
Iteration: 1/1
EXECUTION TIME OF SAWrapSA(CRAID): [1.651 1.609 1.674 1.773 1.73  1.711 1.747 1.715 1.684 1.708 1.678 1.681
 1.73  1.758 1.773 1.709 1.646 1.631 1.681 1.559], MEM [6.414 2.25  0.031 0.008 1.965 0.988 1.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    1.281 0.    0.    0.   ] {'CI': [0.625, 0.617], 'IBS': [0.214, 0.224], 'AUPRC': [0.714, 0.716], 'MSE': [2401710.228, 2699376.688]}
<class 'survivors.tree.decision_tree.CRAID'> {'depth': [10]}
Iteration: 1/1
EXECUTION TIME OF CRAID: [1.937 1.895 2.069 2.101 1.993 1.976 2.032 2.041 2.

In [6]:
df_results

,METHOD,PARAMS,TIME,CI,IBS,AUPRC,MSE,CRIT,TIMES,MEMS,MEM,CI_mean,IBS_mean,AUPRC_mean,MSE_mean
0,ClassifWrapSA(KNeighborsClassifier),{'n_neighbors': 5},1.234413,"[0.5110389610389611, 0.5311688311688312, 0.572...","[0.3177899785972074, 0.23926117737863478, 0.22...","[0.4071347142857144, 0.4892759285714288, 0.417...","[2875195.4971428574, 3070796.2092857147, 22593...",,"[0.23901057243347168, 0.24139070510864258, 0.2...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.544221,0.261694,0.428563,2.672510e+06
1,ClassifWrapSA(RandomForestClassifier),{'n_estimators': 10},1.524835,"[0.5834415584415584, 0.6704545454545454, 0.630...","[0.20812438406816028, 0.17427406454265962, 0.1...","[0.4482053214285715, 0.46070507142857137, 0.46...","[2421587.9942857143, 1739820.9342857148, 23849...",,"[0.24786972999572754, 0.3294212818145752, 0.32...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.626364,0.190776,0.455705,2.142657e+06
2,RegrWrapSA(KNeighborsRegressor),{'n_neighbors': 5},1.414418,"[0.5704545454545454, 0.5581168831168831, 0.608...","[0.42874361542697326, 0.3189030111692633, 0.30...","[0.679910643939394, 0.7332104437229439, 0.7253...","[1424609.1785714286, 1186656.4285714286, 11469...",,"[0.23729300498962402, 0.3060445785522461, 0.28...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.571558,0.358311,0.708013,1.299685e+06
3,SAWrapSA(KaplanMeierFitter),{},1.394307,"[0.5, 0.5, 0.5, 0.5, 0.5]","[0.20329872878125196, 0.18216530965070882, 0.1...","[0.6078425806496679, 0.6005546928734674, 0.607...","[1976689.964480705, 1788654.2615088334, 219485...",,"[0.24472260475158691, 0.27121639251708984, 0.3...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.500000,0.192197,0.605376,1.953460e+06
4,SAWrapSA(CoxPHSurvivalAnalysis),{'alpha': 100},1.678197,"[0.6746753246753247, 0.7032467532467532, 0.684...","[0.1401302204109691, 0.11912412566025657, 0.11...","[0.6891068122578905, 0.6969656045797799, 0.699...","[1842316.951513259, 1577546.2837628557, 176631...",,"[0.3267405033111572, 0.2817802429199219, 0.326...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.668701,0.134738,0.693439,1.781999e+06
5,SAWrapSA(CRAID),{'depth': 10},7.102680,"[0.6451298701298701, 0.662987012987013, 0.6266...","[0.2031049643034096, 0.17609808101774893, 0.22...","[0.7365212149141546, 0.7340941337882605, 0.703...","[2244500.0687337946, 2075063.6883098758, 28734...",,"[1.4453179836273193, 1.3720934391021729, 1.352...","[5.76953125, 0.00390625, 0.0, 0.0078125, 0.011...",5.792969,0.642078,0.209990,0.715095,2.433688e+06
6,CRAID,{'depth': 10},8.274698,"[0.6668831168831169, 0.688961038961039, 0.5922...","[0.21673855633594108, 0.11971061681469919, 0.1...","[0.7335604735367447, 0.797497639607317, 0.7609...","[1027434.0053542962, 745379.9369352116, 127947...",,"[1.6378607749938965, 1.746692180633545, 1.6171...","[0.02734375, 0.0, 0.0, 0.0, 0.0078125]",0.035156,0.639481,0.191457,0.758411,1.069884e+06
7,ClassifWrapSA(KNeighborsClassifier),{'n_neighbors': 5},1.386411,"[0.5110389610389611, 0.5311688311688312, 0.572...","[0.3177899785972074, 0.23926117737863478, 0.22...","[0.4071347142857144, 0.4892759285714288, 0.417...","[2875195.4971428574, 3070796.2092857147, 22593...",,"[0.23865318298339844, 0.24819111824035645, 0.2...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.544221,0.261694,0.428563,2.672510e+06
8,ClassifWrapSA(RandomForestClassifier),{'n_estimators': 10},1.494038,"[0.5613636363636364, 0.6467532467532467, 0.630...","[0.23497124837875813, 0.1591442549339761, 0.17...","[0.46963346428571423, 0.48213321428571404, 0.4...","[2936870.331428572, 1890204.7071428576, 232979...",,"[0.29004549980163574, 0.32977914810180664, 0.2...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.608247,0.196236,0.462848,2.322670e+06
9,RegrWrapSA(KNeighborsRegressor),{'n_neighbors': 5},1.264567,"[0.5704545454545454, 0.5581168831168831, 0.608...","[0.42874361542697326, 0.3189030111692633, 0.30...","[0.679910643939394, 0.7332104437229439, 0.7253...","[1424609.1785714286, 1186656.4285714286, 11469...",,"[0.2384166717529297, 0.2931070327758789, 0.243...","[0.0, 0.0, 0.0, 0.0, 

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
# from sklearn.ensemble import RandomForestClassifier
# from lifelines import KaplanMeierFitter
# from sksurv.linear_model import CoxPHSurvivalAnalysis
# from survivors.tree import CRAID

# experim.add_method(ClassifWrapSA, {"model": [KNeighborsClassifier(n_neighbors=5)]})
# experim.add_method(ClassifWrapSA, {"model": [RandomForestClassifier(n_estimators=10)]})
# experim.add_method(RegrWrapSA, {"model": [KNeighborsRegressor(n_neighbors=5)]})
# experim.add_method(SAWrapSA, {"model": [KaplanMeierFitter()]})
# experim.add_method(SAWrapSA, {"model": [CoxPHSurvivalAnalysis(alpha=100)]})
# experim.add_method(SAWrapSA, {"model": [CRAID(depth=10)]})
# # experim.add_method(RegrWrapSA(KNeighborsRegressor()), {"n_neighbors": [5]})
# # experim.add_method(SAWrapSA(KaplanMeierFitter()), dict())
# # experim.add_method(SAWrapSA(CoxPHSurvivalAnalysis()), {"alpha": [100]})
# # experim.add_method(SAWrapSA(CRAID()), {"depth": [10]})
# experim.run_effective(X, y, verbose=0, stratify_best=[])
# df_results = experim.get_result()